# 2 - Ridge Regression

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso, Ridge, LinearRegression

os.chdir("../")
print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: /Users/gardpavels/uio/fysstk_proj1/FYS-STK4155-Prj1_report


In [2]:
INPUT_DATA = "data/input_data/"  # Path for input data
REPORT_DATA = "data/report_data/" # Path for data ment for the report
REPORT_FIGURES = "figures/" # Path for figures ment for the report

## 2.1 Data
Defining and creating the data


In [3]:
features = 6
#x = np.arange(0, 1, 0.05)
#y = np.arange(0, 1, 0.05)
X = create_X(x, y, n=features) # Design Matrix
z = FrankeFunction(x,y)
z += noise # Adding stochastic noise

NameError: name 'x' is not defined

## 2.3 - Finding optimal lambda for Ridge fit
First we find the optimal value for the lambda parameter by splitting the input data into training and test data. Than we fit the model using different values for lambda. For each lambda value we use the mean square error on the test data evaluate how god the fit is. The best lambda value is used to fit a new model with all data (not using train-test split). <br>
First we find the optimal parameter value of lambda, $\lambda$, by evaluating overfit from evaluation plot where we plot the values for MSE for training and test data

In [ ]:
nbf_lambdas = 100
lambdas = np.logspace(-6,6, nbf_lambdas)
z_train_ridge = pd.DataFrame()
z_hat_train_ridge = pd.DataFrame()
z_test_ridge = pd.DataFrame()
z_hat_test_ridge = pd.DataFrame()

for lam in lambdas:
    #X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False)
    X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False, random_state=4155)
    model = RidgeRegression(lam) # The model
    model.fit(X_train, z_train) # Fitting the model
   
    # Predictions
    z_hat_train = model.predict(X_train) # predict on train data
    z_hat_test = model.predict(X_test) # predict on test data

    # Filling up dataframes
    z_train_ridge[lam] = z_train.flatten() 
    z_hat_train_ridge[lam] = z_hat_train.flatten()
    z_test_ridge[lam] = z_test.flatten()
    z_hat_test_ridge[lam] = z_hat_test.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_ridge - z_hat_train_ridge) ** 2).mean()
mse_scores_test = ((z_test_ridge - z_hat_test_ridge) ** 2).mean()
# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_ridge - z_hat_train_ridge) ** 2).sum() / ((z_train_ridge - z_train_ridge.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_ridge - z_hat_test_ridge) ** 2).sum() / ((z_test_ridge - z_test_ridge.mean())**2).sum()  

# Plots
plt.figure(figsize=(12,8))
plt.plot(-np.log(lambdas), mse_scores_train, label="Training data")
plt.plot(-np.log(lambdas), mse_scores_test, label="Test data")
#plt.plot(mse_scores_train,-np.log(lambdas), label="Training data")
#plt.plot(mse_scores_test,-np.log(lambdas), label="Test data")
plt.xlabel("log(lambda)")
plt.ylabel("MSE")
plt.title("Training evaluation on Ridge regression fit")
plt.legend()
plt.savefig(f"{REPORT_FIGURES}franke_function_Ridge_evaluate_fit.pdf")

## 2.4 Analysis of plots and training metrics
Do the analysis.......

Conclusion:<br>
Based on the analysis, we conclude that a model complexity of degree 5 yields the most optimal fit.

## 2.5 plot of the model using the most optimal parameters

In [ ]:
scale = False

if scale:
    # Data Scalling
    X_scaler = StandardScaler()
    X_scaler.fit(X)
    X_train = X_scaler.transform(X_train)
    z_train = np.expand_dims(z,axis=1)  
    z_scaler = StandardScaler()
    z_scaler.fit(z_train)
    z_train = z_scaler.transform(z_train)
else:
    X_train = X; z_train = z

lam = -1.1
model = RidgeRegression(lam) # The model
model.fit(X_train, z_train) # Fitting the model
z_hat = model.predict(X_train) # predict on train data

# Evaluatation metrics
# TODO:
results_df = pd.DataFrame(columns=["MSE", "R2-score"], index=["Training data", "Test data"])
results_df["MSE"] = MSE(z, z_hat)
results_df["R2-score"] = R2(z, z_hat)
results_df.to_csv(f"{REPORT_DATA}redge_reg_lambda_{lam}.csv")

# Plot
fig = plt.figure()#figsize=(8,8))
ax = plt.axes(projection='3d')
ax.title.set_text(f"Ridge regression fit to the Franke Function\nDegree {degree},$\lambda$:{lam}")
#ax.view_init(elev=5., azim=85.0)
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.scatter3D(x,y,z,c=z, cmap=cm.coolwarm, marker = '.')
plt.savefig(f"{REPORT_FIGURES}franke_function_Ridge_best_fit.pdf")